In [12]:
import numpy as np
from math import log

def createDataSet():
    group = [[1,1,'A'], [1,1,'A'], [1,0,'B'], [0,1,'B'],[0,1,'B']]
    return group

# 计算信息熵
def calcShannonEnt(dataSet):
    count = len(dataSet)
    labelsCount = {}
    for featVec in dataSet:
        feat = featVec[-1]
        if feat not in labelsCount.keys():
            labelsCount[feat] = 0
        labelsCount[feat] += 1
    result = 0.0
    for key in labelsCount:
        prop = float(labelsCount[key] / count)
        result += (-prop)*log(prop,2)
    return result

group = createDataSet()
shannonEnt = calcShannonEnt(group)
print(shannonEnt)

def splitDataSet(dataSet, index, value):
    result = []
    for featVec in dataSet:
        if featVec[index] == value:
            tmpVec = featVec[:index]
            tmpVec.extend(featVec[index+1:])
            result.append(tmpVec)
    return result

#splitDataSet = splitDataSet(group, 1, 1.0)
#print(splitDataSet)

def bestFeatureToSplit(dataSet):
    m = len(dataSet)
    k = len(dataSet[0]) - 1
    shannonEnt = calcShannonEnt(dataSet)
    bestInfoGain = 0
    bestIndex = -1
    for i in range(k):
        featList = [example[i] for example in dataSet]
        featSet = set(featList)
        splitShannonEnt = 0
        for feat in featSet:
            subDataSet = splitDataSet(dataSet,i,feat)
            prop = len(subDataSet) / float(m)
            splitShannonEnt += prop * calcShannonEnt(subDataSet)
            infoGain = shannonEnt - splitShannonEnt
            if infoGain > bestInfoGain:
                bestInfoGain = infoGain
                bestIndex = i
    return bestIndex

bestIndex = bestFeatureToSplit(group)
print(bestIndex)

0.9709505944546686
0
